# TP2 

## Exercicio 1:
    
   #### Considere a descrição da cifra A5/1 que consta no documento +Lógica Computacional: a Cifra A5/1 Informação complementar pode ser obtida no artigo da Wikipedia. 

#### a) Definir e codificar, em z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador.

In [28]:
from z3 import *

# Definir os LFSRs como BitVecs com os tamanhos adequados
LFSR1 = BitVec('LFSR1', 19)
LFSR2 = BitVec('LFSR2', 22)
LFSR3 = BitVec('LFSR3', 23)

# Funções de atualização para cada LFSR
def nextLFSR1(LFSR1):
    f = Extract(18, 18, LFSR1) ^ Extract(17, 17, LFSR1) ^ Extract(16, 16, LFSR1) ^ Extract(13, 13, LFSR1)
    return Concat(f, Extract(18, 1, LFSR1))

def nextLFSR2(LFSR2):
    f = Extract(21, 21, LFSR2) ^ Extract(20, 20, LFSR2)
    return Concat(f, Extract(21, 1, LFSR2))

def nextLFSR3(LFSR3):
    f = Extract(22, 22, LFSR3) ^ Extract(21, 21, LFSR3) ^ Extract(20, 20, LFSR3) ^ Extract(7, 7, LFSR3)
    return Concat(f, Extract(22, 1, LFSR3))

# Bits de clock para cada LFSR
clockBit1 = Extract(8, 8, LFSR1)
clockBit2 = Extract(10, 10, LFSR2)
clockBit3 = Extract(10, 10, LFSR3)

# Função de clock majoritário
def majority(b1, b2, b3):
    return If(b1 + b2 + b3 > 1, BitVecVal(1, 1), BitVecVal(0, 1))

# Calcular o bit majoritário
majority_bit = majority(clockBit1, clockBit2, clockBit3)

# Atualizar os LFSRs com base no bit de clock majoritário
next_LFSR1 = If(clockBit1 == majority_bit, nextLFSR1(LFSR1), LFSR1)
next_LFSR2 = If(clockBit2 == majority_bit, nextLFSR2(LFSR2), LFSR2)
next_LFSR3 = If(clockBit3 == majority_bit, nextLFSR3(LFSR3), LFSR3)


### b) Considere as seguintes propriedades de erro:
### i) Ocorrência de um "burst" 0^t(t-zeros) que ocorre em 2^t passos ou menos.
Tente codificar estas propriedade e cerificar se são acessíveis a partir de um estado inicial aleatoriamente gerado  

In [30]:


# Definir o tamanho de burst e o limite de passos
t = 4  # Número de zeros consecutivos
passos = 2 ** t  # Número máximo de passos

# Configurar o solver
solver = Solver()
curLFSR1 = LFSR1
curLFSR2 = LFSR2
curLFSR3 = LFSR3
countZeros = BitVecVal(0, 32)  # Inicializar o contador de zeros consecutivos

# Loop para simular os passos
for passo in range(passos):
    # Calcular o bit majoritário
    majority_bit = majority(clockBit1, clockBit2, clockBit3)
    
    # Atualizar os LFSRs com base no bit de clock majoritário
    next_LFSR1 = If(clockBit1 == majority_bit, nextLFSR1(curLFSR1), curLFSR1)
    next_LFSR2 = If(clockBit2 == majority_bit, nextLFSR2(curLFSR2), curLFSR2)
    next_LFSR3 = If(clockBit3 == majority_bit, nextLFSR3(curLFSR3), curLFSR3)
    
    # Atualizar o estado atual
    curLFSR1, curLFSR2, curLFSR3 = next_LFSR1, next_LFSR2, next_LFSR3
    
    # Extrair o bit menos significativo para verificar se é zero
    output_bit = Extract(0, 0, curLFSR1) ^ Extract(0, 0, curLFSR2) ^ Extract(0, 0, curLFSR3)
    
    # Checar se o bit é zero
    isZero = output_bit == BitVecVal(0, 1)
    countZeros = If(isZero, countZeros + 1, BitVecVal(0, 32))  # Reinicia se não for zero
    
    # Se atingirmos t zeros consecutivos, adicionamos uma restrição para o solver
    if countZeros == t:
        solver.add(isZero)
        break

# Verificar se conseguimos satisfazer a condição
if solver.check() == sat:
    print("Burst de zeros foi encontrado dentro do limite.")
else:
    print("Não foi encontrado nenhum burst de zeros dentro do limite.")


Burst de zeros foi encontrado dentro do limite.


### ii) Ocorrência de um "burst" de tamanho t que repete um "burst" anterior no mesmo output em 2^(t/2) passos ou menos.
Tente codificar estas propriedade e cerificar se são acessíveis a partir de um estado inicial aleatoriamente gerado  

In [31]:


# Definir o tamanho de burst e o limite de passos
t = 4  # Tamanho do burst
limite_passos = 2 ** (t // 2)  # Número máximo de passos entre bursts repetidos

# Configurar o solver
solver = Solver()
curLFSR1 = LFSR1
curLFSR2 = LFSR2
curLFSR3 = LFSR3

# Lista para armazenar a sequência de bursts
output_history = []

# Loop para simular os passos e verificar repetição de bursts
for passo in range(limite_passos):
    # Calcular o bit majoritário
    majority_bit = majority(clockBit1, clockBit2, clockBit3)
    
    # Atualizar os LFSRs com base no bit de clock majoritário
    next_LFSR1 = If(clockBit1 == majority_bit, nextLFSR1(curLFSR1), curLFSR1)
    next_LFSR2 = If(clockBit2 == majority_bit, nextLFSR2(curLFSR2), curLFSR2)
    next_LFSR3 = If(clockBit3 == majority_bit, nextLFSR3(curLFSR3), curLFSR3)
    
    # Atualizar o estado atual
    curLFSR1, curLFSR2, curLFSR3 = next_LFSR1, next_LFSR2, next_LFSR3
    
    # Extrair o bit menos significativo para formar a saída
    output_bit = Extract(0, 0, curLFSR1) ^ Extract(0, 0, curLFSR2) ^ Extract(0, 0, curLFSR3)
    
    # Adicionar o bit ao histórico
    output_history.append(output_bit)
    
    # Verificar se a sequência de comprimento t se repete
    if len(output_history) >= t:
        burst_sequence = output_history[-t:]  # Últimos t bits
        if any(burst_sequence == output_history[i:i + t] for i in range(len(output_history) - t * 2)):
            # Encontramos uma repetição do burst
            solver.add(output_bit == burst_sequence[0])  # Exemplo de condição de restrição
            break

# Verificar se o solver encontrou uma solução
if solver.check() == sat:
    print("Repetição de burst encontrada dentro do limite de passos.")
else:
    print("Não foi encontrada nenhuma repetição de burst dentro do limite de passos.")


Repetição de burst encontrada dentro do limite de passos.
